In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import show
import re

In [2]:
df = pd.read_csv('data/population_cleaned.csv')

In [3]:
#group the ages by categories :{[0, 18), [18, 40), [40, 65), [65, 100)}
d = pd.DataFrame(columns=['commune', 'sex', 'year', 'nationality', 'age', 'value'])
for l, u in [(0, 18), (18, 40), (40, 65), (65, 100)]:
    tmp = df[df['age'] >= l]
    tmp = tmp[tmp['age'] < u]
    tmp = tmp.groupby(['commune', 'sex', 'year', 'nationality'], as_index=False).sum()
    tmp['age'] = int(u)
    d = pd.concat([d, tmp])
for s in ['year', 'age', 'value']:
    d[s] = d[s].astype(int)
df = d

In [4]:
ids =  pd.read_csv('data/ids')
ids.rename(columns={'name':'commune'}, inplace=True)
ids = ids.sort_values('commune', ascending=True)

In [11]:
def merge(df, year):
    da = df.groupby(['commune','sex', 'nationality', 'age']).sum()
    da = da.unstack(level=0).T
    da.columns = ['%s%s%s' % (a, '%s' % b if b else '', '%s' % c if c else '') for a, b,c in da.columns]
    da.index = da.index.droplevel()
    da = da.reset_index(level=0)
    result = pd.merge(ids, da, left_on='commune', right_on='commune')
    result = result.sort_values('id')
    result['id'] = result['id'].astype(int)
    result.to_csv('data/clean%d.csv' % year, index=False)

In [12]:
for i in range(2010, 2016):
    tmp = df[df['year'] == i]
    tmp = tmp.drop('year', axis=1)
    merge(tmp, i)
    print('year %d: done' % i)

year 2010: done
year 2011: done
year 2012: done
year 2013: done
year 2014: done
year 2015: done
